In [2]:
# start with importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

In [3]:
 # read our csv to df
attrition_df = pd.read_csv("HR-Employee-Attrition.csv")
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [5]:
# there are numerous categorical data cols in this df
# we can use pandas df.dtypes to list all cols wwith categorical data then use OneHotEncoder once on all the categories

# start by generating a categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [6]:
# lets check number of unique values in each col
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [7]:
# none of the above categories has more than 10 unique values
# this means we cna go ahead and encode
# we will encode all at the same time using oneHotEncoder

# start by creating a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [8]:
# lets fit and transform oneHotEncoder using our cat variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

In [9]:
# add encoded names to df
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [10]:
# we will now merge dfs and drop columns
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


#### - Next we split and train our data then we Standardize
#### - first we separate our Model Features (X values) and Target (y values)

In [11]:
# we are trying to predict if one is in the risk of attrition.
# the attrition column is dichotomous so we will keep the YES col (attrition_yes)

# we will first split preprocessed data into features(x) and targets(y)
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# split preprocessed data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# after allocating train and test data, we are ready to standardize numerical data
# we will use StandardScaler object to standardize the numerical features

# we start with a scaler instance
scaler = StandardScaler()

# then fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# we will now create our model - deep neural network
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

# create sequential mode using tf's keras
nn =tf.keras.models.Sequential()

# First hidden layer - add to first hidden layer to the sequential model
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# add second hidden layer to sequential model
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# output layer added to sequential model
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 448       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [19]:
# time to compile our model and define loss and accuracy
# we will use a binary classifier-- binary_crossentropy-- loss function
# we will use adam ---as optimizer
# use accuracy metrics

# compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# we may opt to increase the number of epochs due to complexity of the dataset
# more epochs allow the deep learning model more opportunities to optimize weighted coefficients

# lets train the model eith the increased epochs
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.8276
Epoch 2/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.8276
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.8276
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 6/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 8/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 9/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 10/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4598 - accuracy: 0.8276
Epoch 11/

Epoch 83/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 84/100
35/35 [==============================] - 0s 941us/step - loss: 0.4597 - accuracy: 0.8276
Epoch 85/100
35/35 [==============================] - 0s 942us/step - loss: 0.4597 - accuracy: 0.8276
Epoch 86/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 87/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 88/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 89/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 90/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 91/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8276
Epoch 92/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4597 - accuracy: 0.8

In [22]:
# after training we evaluate the models performance by testing its predictive capability on the test dataset

# evaluating the model using test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 1s - loss: 0.3897 - accuracy: 0.8723
Loss: 0.3896951973438263, Accuracy: 0.8722826242446899


#### -- trying to understand the above evaluation statement

In [23]:
# after training we evaluate the models performance by testing its predictive capability on the test dataset

# evaluating the model using test data
model_loss = nn.evaluate(X_test, y_test, verbose=2)
model_accuracy = nn.evaluate(X_test, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3897 - accuracy: 0.8723
12/12 - 0s - loss: 0.3897 - accuracy: 0.8723
Loss: [0.3896951973438263, 0.8722826242446899], Accuracy: [0.3896951973438263, 0.8722826242446899]


##### - the model was able to predict by correctly identifying employees at risk of attritioon approximately 87% of the time
##### - that is givven by the accuracy metric = 0.8723